## Step 3: Coregistration

In this step, we are going to connect the selected mineral deposits in Step 2 with the trench sample points we have generated in Step 1. The mineral deposits csv file contains only 5 columns -- index, longitude, latitude, age and plate id. These attributes are not enough for machine learning analysis. In order to obtain more data, we need to associate these mineral deposits with the trench sample points. We call this process coregistration.

The coregistration method is simple. For a given mineral deposit, the coregistration process will try to find the nearest trench point within a certain region. If found, the subduction convergence kinematics statistics of the trench point will be associated with the mineral deposit. The attributes retrieved from the trench sample points will be used as input data for machine learning models later.

First, let's run the coregistration script and see what will happen. The coregistration script can be configurated via parameters.py, such as the input mineral deposits file, output file name and region of interest, etc.

Relevant parameters in [parameters.py](parameters.py):

* input_file
* output_dir
* regions
* vector_files
* grid_files


In [1]:
from parameters import parameters as p

#let's print out some of the parameters
#you can change the 'input_file' in parameters.py to use different mineral deposits. 
#Remember the files we have created in step 2?
print('The file name of the mineral deposits: ', p['input_file'])
print('The output folder: ', p['output_dir'])
print('The region of interest(in degree): ', p['regions'])
print('The subduction convergence kinematics statistics file name template: ', p['vector_files'])
print('\n')

import coregistration
#run the coregistration script
coregistration.main()
#some files should have been created at this point
#let's move to the next cell and check the results

The file name of the mineral deposits:  coregistration_input_data_NA_269.csv
The output folder:  coreg_output
The region of interest(in degree):  [5, 10]
The subduction convergence kinematics statistics file name template:  ['./convergence_data/subStats_{time:.2f}.csv']


region of interest: 5
the length of input data is: 28828
region of interest: 10
the length of input data is: 3172
The coregistration output data have been saved in folder coreg_output successfully!
Time taken: 100.05527520179749  seconds


In [ ]:
import pandas as pd
import Utils

#read in the coregistration output file
data = pd.read_csv("./coreg_output/0_vector_subStats.out", header=None) 
display(data.head())#let's print the first 5 rows

print('\nThe meaning of the columns: \n')
Utils.print_columns()

input_data = pd.read_csv(p['input_file'])
display(input_data.head())

#the input data and output data has the same length
print('The shape of the output data: ', data.shape)
print('The shape of the input data: ',input_data.shape)

We can see in above code cell that the input data and output data has the same length. It means, for each input mineral deposit, there is one corresponding data row in the output file. 

The coregistration program takes the mineral deposit coordinates and uses age and plate id to reconstruct the deposits back in time. And then the program searches the nearby subduction trench, if found, copy the subduction convergence kinematics statistics.

##### Now we plot some maps to see the attributes acquired from coregistration.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

def set_ax(ax):
    ax.stock_img()
    ax.set_extent([-170, -90, 10, 70])

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=1, color='gray', alpha=0.5, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xlocator = mticker.FixedLocator([-180, -150, -120, -90, -80, -70, -60,-50,-40,-30, 0, 180])
    gl.ylocator = mticker.FixedLocator([-90,-50,-40, -30, -20,-10, 0, 10, 20, 30, 40,50, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'color': 'gray', 'weight': 'bold', 'fontsize': '5'}
    gl.ylabel_style = {'color': 'gray', 'weight': 'bold', 'fontsize': '5'}

trench_file = './convergence_data/subStats_0.00.csv'
trench_data= np.genfromtxt(trench_file)

import re
deposit_len = int(re.split(r'_|\.', p['input_file'])[-2])

fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={'projection': ccrs.PlateCarree()},figsize=(12,12),dpi=150)
set_ax(ax1)
set_ax(ax2)

cb_1 = ax1.scatter(input_data.loc[:deposit_len,'lon'], input_data.loc[:deposit_len,'lat'], 50, marker='.',
                c=data.loc[:deposit_len,6],  cmap=plt.cm.jet)
cb_2 = ax2.scatter(input_data.loc[:deposit_len,'lon'], input_data.loc[:deposit_len,'lat'], 50, marker='.',
                c=data.loc[:deposit_len,24],  cmap=plt.cm.jet)

ax1.title.set_text('Deposits Coloured By Convergence Rate')
ax2.title.set_text('Deposits Coloured By Subduction Volumn')
cbar_1 = fig.colorbar(cb_1, shrink=0.5, ax=[ax1], orientation='horizontal', pad=0.05)
cbar_1.set_label('Convergence Rate (cm/yr)',size=10)
cbar_1.ax.tick_params(labelsize=7)
cbar_2 = fig.colorbar(cb_2, shrink=0.5, ax=[ax2], orientation='horizontal', pad=0.05)
cbar_2.set_label('Subduction Volumn ($km^3$/yr)',size=10)
cbar_2.ax.tick_params(labelsize=7)
plt.show()

#### This is the end of step 3 and now open the step 4 notebook -- "4_Data_Wrangling.ipynb"